In [ ]:
from datetime import datetime

import camelot
import pandas as pd

In [ ]:
# Configuration
PDF_FILE_PATH = './docs/statement-2.pdf'

# Column Headers from the Document
COLUMN_NAMES = [
    "Trans. Time",
    "Value Date",
    "Description",
    "Debit/Credit(N)",
    "Balance(N)",
    "Channel",
    "Transaction Reference"
]

def is_valid_date(date_string: str) -> bool:
    """Check if a string matches 'YYYY Mon DD HH:MM:SS'."""
    try:
        datetime.strptime(date_string, "%Y %b %d %H:%M:%S")
        return True
    except ValueError:
        return False

def build_transactions_with_validation(raw_df: pd.DataFrame) -> pd.DataFrame:
    """
    The `build_transactions_with_validation` function processes a raw DataFrame of transaction data
    and constructs a cleaned DataFrame of transactions by identifying valid transaction boundaries
    based on date validation. The logic is as follows:

    1. **Initialize Variables**:
        - `transactions`: A list to store completed transactions.
        - `transaction_in_progress`: A dictionary to hold the current transaction being built.
        - `valid_transaction`: A flag to indicate if the current transaction is valid.

    2. **Iterate Through Rows**:
        - For each row, check if the first column contains a valid timestamp using the `is_valid_date` function.
        - If a valid timestamp is found:
            - Append the current `transaction_in_progress` to the `transactions` list (if it exists).
            - Start a new transaction with the current row.
            - Set `valid_transaction` to `True`.
        - If no valid timestamp is found:
            - If no transaction is in progress, initialize `transaction_in_progress` with the current row.
            - Otherwise, concatenate the current row's data into the existing `transaction_in_progress`.
            - If the concatenated data forms a valid timestamp, set `valid_transaction` to `True`.

    3. **Handle Remaining Transaction**:
        - After iterating through all rows, append the last `transaction_in_progress` to the `transactions` list.

    4. **Return Result**:
        - Convert the `transactions` list into a DataFrame and return it.
    """
    transactions = []
    transaction_in_progress = {}
    valid_transaction = False

    display(raw_df.head(50))

    for _, row in raw_df.iterrows():
        timestamp_candidate = str(row.iloc[0]).strip()
        if is_valid_date(timestamp_candidate):
            if transaction_in_progress:
                transactions.append(transaction_in_progress)
            transaction_in_progress = row.to_dict()
            valid_transaction = True
        elif not transaction_in_progress:
            transaction_in_progress = row.to_dict()
        else:
            if valid_transaction:
                merged_date = transaction_in_progress[raw_df.columns[0]] + timestamp_candidate
                if not is_valid_date(merged_date):
                    transactions.append(transaction_in_progress)
                    transaction_in_progress = row.to_dict()
                    valid_transaction = False
                    continue
            for col in raw_df.columns:
                existing_val = transaction_in_progress.get(col, "")
                transaction_in_progress[col] = f"{existing_val}{row[col]}".strip()

            if is_valid_date(transaction_in_progress[raw_df.columns[0]]):
                valid_transaction = True

    if transaction_in_progress:
        transactions.append(transaction_in_progress)

    print(f"\nTotal completed transactions: {len(transactions)}")
    response = pd.DataFrame(transactions)
    display(response.head(50))
    return response

def extract_and_clean_transactions(pdf_path: str) -> pd.DataFrame:
    """Extract, build, and clean transaction data from a PDF."""
    print("Starting extraction...")
    tables = camelot.read_pdf(pdf_path, pages='all', flavor='stream', row_tol=10)
    raw_df = pd.concat([table.df for table in tables], ignore_index=True)
    raw_df.dropna(axis=1, how='all', inplace=True)

    num_columns = min(len(COLUMN_NAMES), raw_df.shape[1])
    raw_df.columns = COLUMN_NAMES[:num_columns]

    print("Building transactions with validation logic...")
    transactions_df = build_transactions_with_validation(raw_df)

    print("Performing final data cleaning...")
    cleaned_df = transactions_df.copy()

    # Clean up newline characters and leading/trailing spaces
    for col in cleaned_df.columns:
        if cleaned_df[col].dtype == 'object':
            cleaned_df.loc[:, col] = (
                cleaned_df[col]
                .str.replace('\n', ' ', regex=False)
                .str.strip()
            )

    def clean_numeric_col(series: pd.Series) -> pd.Series:
        # Remove commas, letters, and any non-numeric characters except for '.' and '-'
        cleaned_series = series.astype(str).str.replace(',', '', regex=False)
        cleaned_series = cleaned_series.str.replace(r'[^\d.-]', '', regex=True)
        return pd.to_numeric(cleaned_series, errors='coerce')

    cleaned_df.loc[:, 'Debit/Credit(N)'] = clean_numeric_col(cleaned_df['Debit/Credit(N)'])
    cleaned_df.loc[:, 'Balance(N)'] = clean_numeric_col(cleaned_df['Balance(N)'])

    cleaned_df.loc[:, 'Trans. Time'] = pd.to_datetime(
        cleaned_df['Trans. Time'],
        format="%Y %b %d %H:%M:%S",
        errors='coerce'
    )
    cleaned_df.loc[:, 'Value Date'] = pd.to_datetime(
        cleaned_df['Value Date'],
        format="%d %b %Y",
        errors='coerce'
    )

    cleaned_df = cleaned_df[cleaned_df['Trans. Time'].notna()]
    cleaned_df.reset_index(drop=True, inplace=True)

    print("Extraction and cleaning complete.")
    return cleaned_df

In [ ]:
if __name__ == '__main__':
    # --- Execute the script ---
    final_transactions_df = extract_and_clean_transactions(PDF_FILE_PATH)

    # --- Display Results ---
    print("\n--- Extracted Transactions ---")
    print(f"Total transactions extracted: {len(final_transactions_df)}")

    # Display the first 10 and last 10 transactions
    print("\nFirst 50 Transactions:")
    display(final_transactions_df.head(50))

    print("\nLast 50 Transactions:")
    display(final_transactions_df.tail(50))

    # --- Optional: Save to CSV ---
    try:
        output_csv_path = f'./output/extracted_transactions{datetime.now().strftime('%d%m%Y%H%M%S')}.csv'
        final_transactions_df.to_csv(output_csv_path, index=False)
        print(f"\nSuccessfully saved cleaned data to '{output_csv_path}'")
    except Exception as e:
        print(f"\nCould not save to CSV: {e}")